
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# トランスフォーマーからの双方向エンコーダー表現 (BERT)

 :ラベル: `sec_bert`

自然言語理解のためにいくつかの単語埋め込みモデルを導入しました。事前トレーニング後の出力は、各行が事前定義された語彙の単語を表すベクトルである行列と考えることができます。実際、これらの単語埋め込みモデルはすべて*コンテキストに依存し*ません。このプロパティを説明することから始めましょう。

## コンテキスト非依存型からコンテキスト依存型へ

:numref: `sec_word2vec_pretraining`と :numref: `sec_synonyms`の実験を思い出してください。たとえば、word2vec と GloVe はどちらも、単語のコンテキスト (存在する場合) に関係なく、同じ事前学習済みベクトルを同じ単語に割り当てます。正式には、任意のトークン $x$ のコンテキストに依存しない表現は、入力として $x$ のみを取る関数 $f(x)$ です。自然言語には多義性と複雑な意味論が豊富にあるため、文脈に依存しない表現には明らかな制限があります。たとえば、「クレーンが飛んでいる」という文脈と「クレーン運転手が来た」という文脈における「クレーン」という言葉は全く異なる意味を持ちます。したがって、同じ単語でも文脈に応じて異なる表現が割り当てられる場合があります。

これは、*単語の表現がその文脈に依存*する、文脈依存型の単語表現の開発を動機づけます。したがって、トークン $x$ のコンテキスト依存表現は、$x$ とそのコンテキスト $c(x)$ の両方に依存する関数 $f(x, c(x))$ になります。一般的なコンテキスト依存表現には、TagLM (言語モデル拡張シーケンス タガー) :cite: `Peters.Ammar.Bhagavatula.ea.2017` 、CoVe (Context Vectors) :cite: `McCann.Bradbury.Xiong.ea.2017` 、および ELMo (言語モデルからの埋め込み) :cite: `Peters.Neumann.Iyyer.ea.2018` 。

たとえば、ELMo はシーケンス全体を入力として受け取り、入力シーケンスから各単語に表現を割り当てる関数です。具体的には、ELMo は、事前トレーニングされた双方向 LSTM からのすべての中間層表現を出力表現として結合します。次に、ELMo 表現と既存モデル内のトークンの元の表現 (GloVe など) を連結するなどして、ELMo 表現が追加機能として下流タスクの既存の教師ありモデルに追加されます。一方で、事前トレーニングされた双方向 LSTM モデル内のすべての重みは、ELMo 表現が追加された後にフリーズされます。一方、既存の教師ありモデルは、特定のタスクに合わせて特別にカスタマイズされています。当時のさまざまなタスクにさまざまな最適なモデルを活用し、ELMo を追加することで、感情分析、自然言語推論、意味役割ラベル付け、共参照解決、固有表現認識、質問応答という 6 つの自然言語処理タスクにわたって最先端の技術が向上しました。

## タスク固有からタスク非依存へ

ELMo は、さまざまな自然言語処理タスクに対するソリューションを大幅に改善しましたが、各ソリューションは依然として*タスク固有の*アーキテクチャに依存しています。ただし、すべての自然言語処理タスクに対して特定のアーキテクチャを作成することは実際には簡単ではありません。 GPT (Generative Pre-Training) モデルは、コンテキスト依存表現のための一般的な*タスク非依存*モデルを設計する取り組みを表しています:cite: `Radford.Narasimhan.Salimans.ea.2018` 。 Transformer デコーダ上に構築された GPT は、テキスト シーケンスを表すために使用される言語モデルを事前トレーニングします。 GPT をダウンストリーム タスクに適用すると、言語モデルの出力が追加の線形出力層に供給され、タスクのラベルが予測されます。事前トレーニングされたモデルのパラメーターをフリーズする ELMo とは対照的に、GPT は、ダウンストリーム タスクの教師あり学習中に、事前トレーニングされた Transformer デコーダーの*すべての*パラメーターを微調整します。 GPT は、自然言語推論、質問応答、文の類似性、分類の 12 のタスクで評価され、モデル アーキテクチャへの最小限の変更でそのうち 9 つのタスクで最先端の技術が向上しました。

ただし、言語モデルの自己回帰的な性質により、GPT は前方 (左から右) のみを参照します。 「現金を預けるために銀行に行った」と「座るために銀行に行った」というコンテキストでは、「銀行」は左側のコンテキストに敏感であるため、GPT は「銀行」に対して同じ表現を返します。にはさまざまな意味があります。

##  BERT: 両方の長所を組み合わせる

これまで見てきたように、ELMo はコンテキストを双方向にエンコードしますが、タスク固有のアーキテクチャを使用します。 GPT はタスクに依存しませんが、コンテキストを左から右にエンコードします。両方の長所を組み合わせた BERT (Bidirectional Encoder Representations from Transformers) はコンテキストを双方向にエンコードし、幅広い自然言語処理タスクに対して最小限のアーキテクチャ変更を必要とします (引用: `Devlin.Chang.Lee.ea.2018` )。事前トレーニング済みの Transformer エンコーダーを使用すると、BERT は双方向コンテキストに基づいて任意のトークンを表現できます。下流タスクの教師あり学習中、BERT は 2 つの点で GPT に似ています。まず、BERT 表現は、トークンごとの予測かシーケンス全体の予測など、タスクの性質に応じてモデル アーキテクチャに最小限の変更を加えて、追加された出力層に供給されます。次に、事前トレーニングされた Transformer エンコーダーのすべてのパラメーターが微調整され、追加の出力レイヤーは最初からトレーニングされます。 :numref: `fig_elmo-gpt-bert` ELMo、GPT、BERT の違いを示しています。

![](../img/elmo-gpt-bert.svg) :label: `fig_elmo-gpt-bert`

 BERT は、(i) 単一テキストの分類 (感情分析など)、(ii) テキスト ペアの分類 (自然言語推論など)、(iii) 質問応答という広範なカテゴリの下で 11 の自然言語処理タスクに関する最先端技術をさらに改善しました。 (iv) テキストのタグ付け (例: 固有表現認識)。コンテキスト依存型 ELMo からタスク非依存型 GPT および BERT まで、2018 年に提案されたすべての手法は、概念的にはシンプルでありながら経験的に強力な自然言語の深い表現の事前トレーニングであり、さまざまな自然言語処理タスクに対するソリューションに革命をもたらしました。

この章の残りの部分では、BERT の事前トレーニングについて詳しく説明します。 :numref: `chap_nlp_app`で自然言語処理アプリケーションを説明する際に、ダウンストリーム アプリケーション向けの BERT の微調整について説明します。


In [1]:
import torch
from torch import nn
from d2l import torch as d2l


## [**入力表現**]

 :label: `subsec_bert_input_rep`

自然言語処理では、一部のタスク (感情分析など) は単一のテキストを入力として受け取りますが、他のタスク (自然言語推論など) では、入力はテキスト シーケンスのペアになります。 BERT 入力シーケンスは、単一のテキストとテキストのペアの両方を明確に表します。前者では、BERT 入力シーケンスは、特別な分類トークン「&lt;cls&gt;」、テキスト シーケンスのトークン、および特別な分離トークン「&lt;sep&gt;」の連結です。後者では、BERT 入力シーケンスは、最初のテキスト シーケンスのトークン「&lt;cls&gt;」、2 番目のテキスト シーケンスのトークン「&lt;sep&gt;」、および「&lt;sep&gt;」を連結したものです。 「BERT 入力シーケンス」という用語を他のタイプの「シーケンス」と一貫して区別します。たとえば、1 つの*BERT 入力シーケンスには、* 1 つの*テキスト シーケンス*または 2 つの*テキスト シーケンス*が含まれる場合があります。

テキストのペアを区別するために、学習されたセグメントの埋め込み $\mathbf{e}_A$ と $\mathbf{e}_B$ が、それぞれ最初のシーケンスと 2 番目のシーケンスのトークン埋め込みに追加されます。単一のテキスト入力の場合、$\mathbf{e}_A$ のみが使用されます。

次の`get_tokens_and_segments` 、1 文または 2 文を入力として受け取り、BERT 入力シーケンスのトークンとそれらに対応するセグメント ID を返します。


In [2]:
#@save
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """Get tokens of the BERT input sequence and their segment IDs."""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0 and 1 are marking segment A and B, respectively
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments


BERT は、双方向アーキテクチャとして Transformer エンコーダを選択します。 Transformer エンコーダでは一般的に、位置埋め込みが BERT 入力シーケンスのすべての位置に追加されます。ただし、元の Transformer エンコーダとは異なり、BERT は*学習可能な*位置埋め込みを使用します。要約すると、:numref: `fig_bert-input` 、BERT 入力シーケンスの埋め込みがトークン 埋め込み、セグメント 埋め込み、および位置埋め込みの合計であることを示します。 

![](../img/bert-input.svg) :label: `fig_bert-input`

次の [ **`BERTEncoder` class** ] は、 :numref: `sec_transformer`に実装されている`TransformerEncoder`クラスに似ています。 `TransformerEncoder`とは異なり、 `BERTEncoder`セグメント エンベディングと学習可能な位置エンベディングを使用します。


In [3]:
#@save
class BERTEncoder(nn.Module):
    """BERT encoder."""
    def __init__(self, vocab_size, num_hiddens, ffn_num_hiddens, num_heads,
                 num_blks, dropout, max_len=1000, **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)
        self.segment_embedding = nn.Embedding(2, num_hiddens)
        self.blks = nn.Sequential()
        for i in range(num_blks):
            self.blks.add_module(f"{i}", d2l.TransformerEncoderBlock(
                num_hiddens, ffn_num_hiddens, num_heads, dropout, True))
        # In BERT, positional embeddings are learnable, thus we create a
        # parameter of positional embeddings that are long enough
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len,
                                                      num_hiddens))

    def forward(self, tokens, segments, valid_lens):
        # Shape of `X` remains unchanged in the following code snippet:
        # (batch size, max sequence length, `num_hiddens`)
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        X = X + self.pos_embedding[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X


語彙サイズが 10000 であるとします**`BERTEncoder`の推論**を進めるために、そのインスタンスを作成し、そのパラメーターを初期化しましょう。


In [4]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
ffn_num_input, num_blks, dropout = 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, ffn_num_hiddens, num_heads,
                      num_blks, dropout)


`tokens`長さ 8 の 2 つの BERT 入力シーケンスとして定義します。各トークンは語彙のインデックスです。入力`tokens`を使用した`BERTEncoder`の前方推論は、各トークンがハイパーパラメーター`num_hiddens`によって事前定義された長さのベクトルによって表されるエンコードされた結果を返します。このハイパーパラメータは通常、Transformer エンコーダの*隠れサイズ*(隠れユニットの数) と呼ばれます。


In [5]:
tokens = torch.randint(0, vocab_size, (2, 8))
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
encoded_X = encoder(tokens, segments, None)
encoded_X.shape

torch.Size([2, 8, 768])


## 事前トレーニングタスク

:label: `subsec_bert_pretraining_tasks`

 `BERTEncoder`の前方推論により、入力テキストの各トークンと挿入された特別なトークン「&lt;cls&gt;」の BERT 表現が得られます。および「&lt;seq&gt;」。次に、これらの表現を使用して、BERT を事前トレーニングするための損失関数を計算します。事前トレーニングは、マスクされた言語モデリングと次の文の予測という 2 つのタスクで構成されます。

###  [**マスクされた言語モデリング**]

 :label: `subsec_mlm`

 :numref: `sec_language-model`に示すように、言語モデルは左側のコンテキストを使用してトークンを予測します。各トークンを表すためにコンテキストを双方向にエンコードするために、BERT はトークンをランダムにマスクし、双方向コンテキストからのトークンを使用してマスクされたトークンを自己教師付きの方法で予測します。このタスクは、*マスクされた言語モデル*と呼ばれます。

この事前トレーニング タスクでは、予測用のマスクされたトークンとしてトークンの 15% がランダムに選択されます。ラベルを使用して不正行為を行わずにマスクされたトークンを予測するための 1 つの簡単なアプローチは、常に特別な「&amp;lt;mask&amp;gt;」に置き換えることです。 BERT 入力シーケンス内のトークン。しかし、人工的な特殊トークン「＜マスク＞」は、微調整では表示されません。事前トレーニングと微調整の間のこのような不一致を避けるために、トークンが予測のためにマスクされている場合 (たとえば、「この映画は素晴らしい」で「素晴らしい」がマスクされて予測されるように選択されている場合)、入力では次のように置き換えられます。
- 特別な「&lt;マスク&gt;」 80% の確率でトークンが使用されます (たとえば、「この映画は素晴らしいです」は「この映画は &lt;マスク&gt;」になります)。
-  10% の確率でランダムなトークン (たとえば、「この映画は素晴らしいです」が「この映画は飲み物です」になります)。
- ラベル トークンは 10% の確率で変更されません (たとえば、「この映画は素晴らしいです」は「この映画は素晴らしいです」になります)。

 15% の時間のうち 10% の間、ランダムなトークンが挿入されることに注意してください。この時折発生するノイズにより、BERT は双方向コンテキスト エンコーディングにおいて、マスクされたトークン (特にラベル トークンが変更されていない場合) への偏りが少なくなります。

 BERT 事前トレーニングのマスクされた言語モデル タスクでマスクされたトークンを予測するために、次の`MaskLM`クラスを実装します。予測には 1 隠れ層 MLP ( `self.mlp` ) が使用されます。前方推論では、 `BERTEncoder`のエンコード結果と予測用のトークン位置の 2 つの入力を受け取ります。出力は、これらの位置での予測結果です。


In [6]:
#@save
class MaskLM(nn.Module):
    """The masked language model task of BERT."""
    def __init__(self, vocab_size, num_hiddens, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        self.mlp = nn.Sequential(nn.LazyLinear(num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.LazyLinear(vocab_size))

    def forward(self, X, pred_positions):
        num_pred_positions = pred_positions.shape[1]
        pred_positions = pred_positions.reshape(-1)
        batch_size = X.shape[0]
        batch_idx = torch.arange(0, batch_size)
        # Suppose that `batch_size` = 2, `num_pred_positions` = 3, then
        # `batch_idx` is `torch.tensor([0, 0, 0, 1, 1, 1])`
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)
        masked_X = X[batch_idx, pred_positions]
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat


[ **`MaskLM`の前方推論**] を実証するために、そのインスタンス`mlm`作成し、初期化します。 `BERTEncoder`の前方推論からの`encoded_X` 2 つの BERT 入力シーケンスを表すことを思い出してください。 `encoded_X`のいずれかの BERT 入力シーケンスで予測する 3 つのインデックスとして`mlm_positions`を定義します。 `mlm`の前方推論は、 `encoded_X`のすべてのマスクされた位置`mlm_positions`での予測結果`mlm_Y_hat`を返します。各予測の結果のサイズは語彙のサイズと等しくなります。


In [7]:
mlm = MaskLM(vocab_size, num_hiddens)
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
mlm_Y_hat = mlm(encoded_X, mlm_positions)
mlm_Y_hat.shape

torch.Size([2, 3, 10000])


マスクの下の予測トークン`mlm_Y_hat`のグラウンド トゥルース ラベル`mlm_Y`を使用すると、BERT 事前トレーニングでマスクされた言語モデル タスクのクロス エントロピー損失を計算できます。


In [8]:
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
loss = nn.CrossEntropyLoss(reduction='none')
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l.shape

torch.Size([6])


### [**次の文の予測**]

 :label: `subsec_nsp`

マスクされた言語モデリングは、単語を表現するための双方向コンテキストをエンコードできますが、テキストのペア間の論理関係を明示的にモデル化するわけではありません。 2 つのテキスト シーケンス間の関係を理解するために、BERT は事前トレーニングでバイナリ分類タスクで*ある次の文の予測*を考慮します。事前トレーニング用に文のペアを生成する場合、半分の時間は実際に「True」というラベルが付いた連続した文になります。一方、残りの半分の時間では、2 番目の文が「False」というラベルが付いたコーパスからランダムにサンプリングされます。

次の`NextSentencePred`クラスは、1 隠れ層 MLP を使用して、2 番目の文が BERT 入力シーケンスの最初の文の次の文であるかどうかを予測します。 Transformer エンコーダのセルフアテンションにより、特別なトークン「&amp;lt;cls&amp;gt;」の BERT 表現は、入力からの 2 つの文の両方をエンコードします。したがって、MLP 分類器の出力層 ( `self.output` ) は`X`入力として受け取ります。ここで、 `X`は、入力がエンコードされた「&lt;cls&gt;」である MLP 隠れ層の出力です。トークン。


In [9]:
#@save
class NextSentencePred(nn.Module):
    """The next sentence prediction task of BERT."""
    def __init__(self, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        self.output = nn.LazyLinear(2)

    def forward(self, X):
        # `X` shape: (batch size, `num_hiddens`)
        return self.output(X)


[ **`NextSentencePred`の前方推論**] インスタンスが、各 BERT 入力シーケンスのバイナリ予測を返すことがわかります。


In [10]:
# PyTorch by default will not flatten the tensor as seen in mxnet where, if
# flatten=True, all but the first axis of input data are collapsed together
encoded_X = torch.flatten(encoded_X, start_dim=1)
# input_shape for NSP: (batch size, `num_hiddens`)
nsp = NextSentencePred()
nsp_Y_hat = nsp(encoded_X)
nsp_Y_hat.shape

torch.Size([2, 2])


2 つのバイナリ分類のクロスエントロピー損失も計算できます。


In [11]:
nsp_y = torch.tensor([0, 1])
nsp_l = loss(nsp_Y_hat, nsp_y)
nsp_l.shape

torch.Size([2])


前述の両方の事前トレーニング タスクのすべてのラベルは、手動でのラベル付けの労力を必要とせずに、事前トレーニング コーパスから簡単に取得できることは注目に値します。オリジナルの BERT は、BookCorpus :cite: `Zhu.Kiros.Zemel.ea.2015`と英語版 Wikipedia を連結して事前トレーニングされています。これら 2 つのテキスト コーパスは膨大で、それぞれ 8 億語と 25 億語あります。

##  [**すべてをまとめる**]

 BERT を事前トレーニングする場合、最終的な損失関数は、マスクされた言語モデリングと次の文の予測の両方の損失関数の線形結合になります。これで、 `BERTEncoder` 、 `MaskLM` 、 `NextSentencePred`の 3 つのクラスをインスタンス化することで`BERTModel`クラスを定義できるようになりました。前方推論は、エンコードされた BERT 表現`encoded_X` 、マスクされた言語モデリングの予測`mlm_Y_hat` 、および次の文の予測`nsp_Y_hat`を返します。


In [12]:
#@save
class BERTModel(nn.Module):
    """The BERT model."""
    def __init__(self, vocab_size, num_hiddens, ffn_num_hiddens,
                 num_heads, num_blks, dropout, max_len=1000):
        super(BERTModel, self).__init__()
        self.encoder = BERTEncoder(vocab_size, num_hiddens, ffn_num_hiddens,
                                   num_heads, num_blks, dropout,
                                   max_len=max_len)
        self.hidden = nn.Sequential(nn.LazyLinear(num_hiddens),
                                    nn.Tanh())
        self.mlm = MaskLM(vocab_size, num_hiddens)
        self.nsp = NextSentencePred()

    def forward(self, tokens, segments, valid_lens=None, pred_positions=None):
        encoded_X = self.encoder(tokens, segments, valid_lens)
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # The hidden layer of the MLP classifier for next sentence prediction.
        # 0 is the index of the '<cls>' token
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :]))
        return encoded_X, mlm_Y_hat, nsp_Y_hat


## まとめ
- word2vec や GloVe などの Word 埋め込みモデルはコンテキストに依存しません。単語のコンテキスト (存在する場合) に関係なく、同じ事前学習済みベクトルを同じ単語に割り当てます。彼らにとって、自然言語の多義性や複雑な意味論を適切に扱うのは困難です。
-  ELMo や GPT などの文脈依存の単語表現の場合、単語の表現はその文脈に依存します。
-  ELMo はコンテキストを双方向にエンコードしますが、タスク固有のアーキテクチャを使用します (ただし、すべての自然言語処理タスクに対して特定のアーキテクチャを作成することは実際には簡単ではありません)。 GPT はタスクに依存しませんが、コンテキストを左から右にエンコードします。
-  BERT は両方の長所を組み合わせています。コンテキストを双方向にエンコードし、幅広い自然言語処理タスクに対して最小限のアーキテクチャ変更を必要とします。
-  BERT 入力シーケンスの埋め込みは、トークン埋め込み、セグメント埋め込み、および位置埋め込みの合計です。
-  BERT の事前トレーニングは、マスクされた言語モデリングと次の文の予測という 2 つのタスクで構成されます。前者は単語を表現するための双方向コンテキストをエンコードできますが、後者はテキストのペア間の論理関係を明示的にモデル化します。

## 演習
1. 他のすべての条件が等しい場合、マスクされた言語モデルが収束するために必要な事前トレーニング ステップは、左から右への言語モデルよりも多いか少ないでしょうか?なぜ？
1.  BERT の元の実装では、 `BERTEncoder`の位置方向フィードフォワード ネットワーク ( `d2l.TransformerEncoderBlock`経由) と`MaskLM`の全結合層は両方ともガウス誤差線形単位 (GELU) を使用します:cite: `Hendrycks.Gimpel.2016`を活性化関数として。 GELUとReLUの違いについての研究。



[ディスカッション](https://discuss.d2l.ai/t/1490)
